In [88]:
#create dictionaries of data with exposure as keys
stdDict2 = {}
meanDict2 = {}
varDict2 = {}

In [90]:
#setup
from FrontPanelAPI import ok
import time
import numpy as np
import cv2

dev = ok.okCFrontPanel()
deviceCount = dev.GetDeviceCount()
for i in range(deviceCount):
    print 'Device[{0}] Model: {1}'.format(i, dev.GetDeviceListModel(i))
    print 'Device[{0}] Serial: {1}'.format(i, dev.GetDeviceListSerial(i))

dev = ok.okCFrontPanel()
dev.OpenBySerial("")
dev.ConfigureFPGA("ok_imager.bit")

Device[0] Model: 21
Device[0] Serial: 143400091C


0

In [72]:
#please close aperture

In [74]:
#grab 100 frames with aperture closed to obtain offset

if dev.IsFrontPanelEnabled():
    row = 160
    N_adc = 4
    N_adcCh = 3
    N_mux = 46
    col = N_adc*N_adcCh*N_mux
    datain128 = bytearray(262144)
    datain1 = bytearray(88320)
    im = np.zeros((row ,col), np.uint8)
    im1 = np.zeros((79 ,60), np.uint8)
    im2 = np.zeros((79 ,60), np.uint8)
    
    exp =3
    exposure = 10**exp
    
    while exp<4.9:
        offset1 = np.zeros((79,60))
        elsecounter = 0   
        rep = 0
        dev.SetWireInValue(0x11,int(exposure))
        dev.SetWireInValue(0x12,1)
        dev.SetWireInValue(0x13,1)
        dev.SetWireInValue(0x14,0)
        dev.SetWireInValue(0x15, 0xfff003ff)
        dev.UpdateWireIns()
        # assert reset signal to initialize the FIFO
        dev.SetWireInValue(0x10, 0xff, 0x01)
        dev.UpdateWireIns()
        # deactivate reset signal and activate counter.
        dev.SetWireInValue(0x10, 0x00, 0x01)
        dev.UpdateWireIns()
        time.sleep(0.01)
        while rep<120:
            dev.UpdateTriggerOuts()
            # If the FIFO is full, read everything and display one frame only
            if dev.IsTriggered(0x6A, 0x01) == True:
                dev.ReadFromPipeOut(0xA0, datain128)
                for i in range(row):
                    for j in range(N_adc):
                        for k in range(N_adcCh):
                            for l in range(N_mux):
                                im[row-1-i][col-1-(j*N_adcCh*N_mux+(2-k)*N_mux+45-l)] = datain128[i*col+l*N_adc*N_adcCh+k*N_adc+j]
                #im = im/255
                im1[:,:] = im[1:80,143:263:2]
                if np.any(im1):
                    rep+=1
                    if rep>20: #to account for noise at the beginning
                        offset1 +=im1

            # If one frame is ready in FIFO
            elif dev.IsTriggered(0x6A, 0x02) == True:
                dev.ReadFromPipeOut(0xA0, datain1)
                for i in range(row):
                    for j in range(N_adc):
                        for k in range(N_adcCh):
                            for l in range(N_mux):
                                im[row-1-i][col-1-(j*N_adcCh*N_mux+(2-k)*N_mux+45-l)] = datain1[i*col+l*N_adc*N_adcCh+k*N_adc+j]
                #im = im/255
                im1[:,:] = im[1:80,143:263:2]
                rep+=1
                if np.any(im1):
                    if rep>20: #to account for noise at the beginning
                        offset1 +=im1
            else:
                elsecounter += 1
                print elsecounter
                if elsecounter>10:
                    elsecounter = 0
                    # assert reset signal to initialize the FIFO
                    dev.SetWireInValue(0x10, 0xff, 0x01)
                    dev.UpdateWireIns()
                    # deactivate reset signal and activate counter.
                    dev.SetWireInValue(0x10, 0x00, 0x01)
                    dev.UpdateWireIns()

            if (rep)%10 ==0:
                print rep 
        offset1= offset1/100
        cv2.imwrite(str(int(exposure))+'_2.png',offset1) #save offset images
        exp +=0.1
        exposure = 10**exp
else:
    print("FrontPanel host interface not detected.")

10
20
30
40
50
60
70
80
90
100
110
120
10
20
30
40
50
60
70
80
90
100
110
120
10
20
30
40
50
60
70
80
90
100
110
120
10
20
30
40
50
60
70
80
90
100
110
120
10
20
30
40
50
60
70
80
90
100
110
120
10
20
30
40
50
60
70
80
90
100
110
120
10
20
30
40
50
60
70
80
90
100
110
120
10
20
30
40
50
60
70
80
90
100
110
120
1
10
20
30
40
50
60
70
80
90
100
110
120
10
20
30
40
50
60
70
80
90
100
110
120
1
10
20
30
40
50
60
70
80
90
100
110
120
0
1
0
10
20
30
40
50
60
70
80
90
100
110
120
0
10
20
30
40
50
60
70
80
90
100
110
120
0
10
20
30
40
50
60
70
80
90
100
110
120
0
1
0
10
20
30
40
50
60
70
80
90
100
110
120
0
10
20
30
40
50
60
70
80
90
100
110
120
0
10
20
30
40
50
60
70
80
90
100
110
120
0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
0
12
0
0
13
0
0
14
0
0
15
0
0
16
0
0
17
0
0
18
0
0
19
0
0
20
0
0
21
0
0
10
20
30
40
50
60
70
80
90
100
110
120
0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
0
12
0
0
13
0
0
14
0
0
10
20
30
40
50
60
70
80
90
100
110
120
0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
1

10
920
10
10
921
10
10
922
10
10
923
10
10
924
10
10
925
10
10
926
10
10
927
10
10
928
10
10
929
10
10
930
10
10
931
10
10
932
10
10
933
10
10
934
10
10
935
10
10
936
10
10
937
10
10
938
10
10
939
10
10
940
10
10
941
10
10
942
10
10
943
10
10
944
10
10
945
10
10
946
10
10
947
10
10
948
10
10
949
10
10
950
10
10
951
10
10
952
10
10
953
10
10
954
10
10
955
10
10
956
10
10
957
10
10
958
10
10
959
10
10
960
10
10
961
10
10
962
10
10
963
10
10
964
10
10
965
10
10
966
10
10
967
10
10
968
10
10
969
10
10
970
10
10
971
10
10
972
10
10
973
10
10
974
10
10
975
10
10
976
10
10
977
10
10
978
10
10
979
10
10
980
10
10
981
10
10
982
10
10
983
10
10
984
10
10
985
10
10
986
10
10
987
10
10
988
10
10
989
10
10
990
10
10
991
10
10
992
10
10
993
10
10
994
10
10
995
10
10
996
10
10
997
10
10
998
10
10
999
10
10
1000
10
10
1001
10
10
1002
10
10
1003
10
10
1004
10
10
1005
10
10
1006
10
10
1007
10
10
1008
10
10
1009
10
10
1010
10
10
1011
10
10
1012
10
10
1013
10
10
1014
10
10
1015
10
10
1016
10
10
1017
10
10

KeyboardInterrupt: 

In [84]:
#if image did not turn out well, do that one manually
exposures = [1995, 3162,2511,10000, 100000]
for exposure in exposures:
    offset1 = np.zeros((79,60))
    elsecounter = 0   
    rep = 0
    dev.SetWireInValue(0x11,int(exposure))
    dev.SetWireInValue(0x12,1)
    dev.SetWireInValue(0x13,1)
    dev.SetWireInValue(0x14,0)
    dev.SetWireInValue(0x15, 0xfff003ff)
    dev.UpdateWireIns()
    # assert reset signal to initialize the FIFO
    dev.SetWireInValue(0x10, 0xff, 0x01)
    dev.UpdateWireIns()
    # deactivate reset signal and activate counter.
    dev.SetWireInValue(0x10, 0x00, 0x01)
    dev.UpdateWireIns()
    time.sleep(0.01)
    while rep<120:
        dev.UpdateTriggerOuts()
        # If the FIFO is full, read everything and display one frame only
        if dev.IsTriggered(0x6A, 0x01) == True:
            dev.ReadFromPipeOut(0xA0, datain128)
            for i in range(row):
                for j in range(N_adc):
                    for k in range(N_adcCh):
                        for l in range(N_mux):
                            im[row-1-i][col-1-(j*N_adcCh*N_mux+(2-k)*N_mux+45-l)] = datain128[i*col+l*N_adc*N_adcCh+k*N_adc+j]
            #im = im/255
            im1[:,:] = im[1:80,143:263:2]
            if np.any(im1>100):
                # assert reset signal to initialize the FIFO
                dev.SetWireInValue(0x10, 0xff, 0x01)
                dev.UpdateWireIns()
                # deactivate reset signal and activate counter.
                dev.SetWireInValue(0x10, 0x00, 0x01)
                dev.UpdateWireIns() 
            elif np.any(im1):
                rep+=1
                if rep>20: #to account for noise at the beginning
                    offset1 +=im1

        # If one frame is ready in FIFO
        elif dev.IsTriggered(0x6A, 0x02) == True:
            dev.ReadFromPipeOut(0xA0, datain1)
            for i in range(row):
                for j in range(N_adc):
                    for k in range(N_adcCh):
                        for l in range(N_mux):
                            im[row-1-i][col-1-(j*N_adcCh*N_mux+(2-k)*N_mux+45-l)] = datain1[i*col+l*N_adc*N_adcCh+k*N_adc+j]
            #im = im/255
            im1[:,:] = im[1:80,143:263:2]
            if np.any(im1>100):
                # assert reset signal to initialize the FIFO
                dev.SetWireInValue(0x10, 0xff, 0x01)
                dev.UpdateWireIns()
                # deactivate reset signal and activate counter.
                dev.SetWireInValue(0x10, 0x00, 0x01)
                dev.UpdateWireIns() 
            elif np.any(im1):
                rep+=1
                if rep>20: #to account for noise at the beginning
                    offset1 +=im1
        else:
            elsecounter += 1
            print elsecounter
            if elsecounter>10:
                elsecounter =0
                # assert reset signal to initialize the FIFO
                dev.SetWireInValue(0x10, 0xff, 0x01)
                dev.UpdateWireIns()
                # deactivate reset signal and activate counter.
                dev.SetWireInValue(0x10, 0x00, 0x01)
                dev.UpdateWireIns()

        if (rep)%10 ==0:
            print rep 
    offset1= offset1/100
    cv2.imwrite(str(exposure)+'.png',offset1) #save offset images

KeyboardInterrupt: 

In [47]:
#please open aperture and expose camera to even illumination


[[34 36 33 ..., 29 32 31]
 [41 34 34 ..., 31 32 32]
 [33 32 35 ..., 31 28 33]
 ..., 
 [35 34 36 ..., 28 38 31]
 [33 35 35 ..., 30 35 28]
 [34 33 34 ..., 29 32 29]]


In [2]:
#grab 100 frames with aperture open to obtain signal 
if dev.IsFrontPanelEnabled():
    row = 160
    N_adc = 4
    N_adcCh = 3
    N_mux = 46
    col = N_adc*N_adcCh*N_mux
    datain128 = bytearray(262144)
    datain1 = bytearray(88320)
    im = np.zeros((row ,col), np.uint8)
    im1 = np.zeros((79 ,60), np.uint8)
    im2 = np.zeros((79 ,60), np.uint8)
    
    exp =3
    exposure = 10**exp
    
    while exp<4.9:
        data1 = np.zeros((100,79,60))
        imgcheck = np.zeros((79,60))
        elsecounter = 0 
        rep = 0
        offset1 = cv2.imread(str(int(exposure))+'.png',0)
        dev.SetWireInValue(0x11,int(exposure))
        dev.SetWireInValue(0x12,1)
        dev.SetWireInValue(0x13,0)
        dev.SetWireInValue(0x14,0)
        dev.SetWireInValue(0x15, 0xfff003ff)
        dev.UpdateWireIns()
        # assert reset signal to initialize the FIFO
        dev.SetWireInValue(0x10, 0xff, 0x01)
        dev.UpdateWireIns()
        # deactivate reset signal and activate counter.
        dev.SetWireInValue(0x10, 0x00, 0x01)
        dev.UpdateWireIns()
        time.sleep(0.01)
        while rep<120:
            dev.UpdateTriggerOuts()
            # If the FIFO is full, read everything and display one frame only
            if dev.IsTriggered(0x6A, 0x01) == True:
                dev.ReadFromPipeOut(0xA0, datain128)
                for i in range(row):
                    for j in range(N_adc):
                        for k in range(N_adcCh):
                            for l in range(N_mux):
                                im[row-1-i][col-1-(j*N_adcCh*N_mux+(2-k)*N_mux+45-l)] = datain128[i*col+l*N_adc*N_adcCh+k*N_adc+j]
                
                if np.any(im1):
                    im1[:,:] = im[1:80,142:262:2] - offset1
                    rep+=1
                    if rep>20: #to account for noise at the beginning
                        imgcheck+=im1
                        data1[rep-20] =im1

            # If one frame is ready in FIFO
            elif dev.IsTriggered(0x6A, 0x02) == True:
                dev.ReadFromPipeOut(0xA0, datain1)
                for i in range(row):
                    for j in range(N_adc):
                        for k in range(N_adcCh):
                            for l in range(N_mux):
                                im[row-1-i][col-1-(j*N_adcCh*N_mux+(2-k)*N_mux+45-l)] = datain1[i*col+l*N_adc*N_adcCh+k*N_adc+j]
                
                if np.any(im1):
                    im1[:,:] = im[1:80,142:262:2] -offset1
                    rep+=1
                    if rep>20: #to account for noise at the beginning
                        imgcheck+=im1
                        data1[rep-20] =im1
            else:
                elsecounter += 1
                print elsecounter
                if elsecounter>10:
                    elsecounter = 0
                    # assert reset signal to initialize the FIFO
                    dev.SetWireInValue(0x10, 0xff, 0x01)
                    dev.UpdateWireIns()
                    # deactivate reset signal and activate counter.
                    dev.SetWireInValue(0x10, 0x00, 0x01)
                    dev.UpdateWireIns()

            if (rep)%10 ==0 and rep!=0:
                print rep 
        #compute the standard deviation and average signal of each pixel 
        imgcheck = imgcheck/100 #make sure the data is ok
        cv2.imwrite(str(int(exposure))+'test.png', imgcheck)
        pixstd1 = np.zeros((79,60))
        pixmean1 = np.zeros((79,60))
        pixvar1 = np.zeros((79,60))
        for row in range(79):
            for col in range(60):
                pixstd1[row,col] = np.std(data1[:,row,col])
                pixmean1[row,col] = np.mean(data1[:,row,col])
                pixvar1[row,col] = np.var(data1[:,row,col])
        stdDict[int(exposure)] = pixstd1
        meanDict[int(exposure)] = pixmean1
        varDict[int(exposure)] = pixvar1
        exp +=0.1
        exposure = 10**exp
else:
    print("FrontPanel host interface not detected.")

Device[0] Model: 21
Device[0] Serial: 143400091C


KeyboardInterrupt: 

In [ ]:
#save dictionaries
with open('stdDict2.pkl', 'wb') as output:
    pickle.dump(stdDict2, output)

with open('meanDict2.pkl', 'wb') as output:
    pickle.dump(meanDict2, output)
    
with open('varDict2.pkl', 'wb') as output:
    pickle.dump(varDict2, output)    